# SVM Classifier with ZKML

## Create and export model via TF -> TFLite

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys

    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sk2torch"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import json
import numpy as np
from sklearn import datasets
import sk2torch
import torch
import ezkl
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
sys.path.insert(0, "../../..")
from zkml.python.converter import Converter

CKPT_PATH = os.path.join(os.getcwd(), "svm")
TFLITE_PATH = os.path.join(os.getcwd(), "svm.tflite")
OUT_MODEL_PATH = os.path.join(os.getcwd(), "svm.msgpack")
OUT_INPUT_NPY_PATH = os.path.join(os.getcwd(), "svm_input.npy")
OUT_INPUT_PATH = os.path.join(os.getcwd(), "svm_input.msgpack")
OUT_CONFIG_PATH = os.path.join(os.getcwd(), "svm_config.json")
iris = datasets.load_iris()
x_vals = np.array([[x[0], x[3]] for x in iris.data])
y_vals = np.array([1 if y == 0 else -1 for y in iris.target])
setosa_x = [d[1] for i, d in enumerate(x_vals) if y_vals[i] == 1]
setosa_y = [d[0] for i, d in enumerate(x_vals) if y_vals[i] == 1]
not_setosa_x = [d[1] for i, d in enumerate(x_vals) if y_vals[i] == -1]
not_setosa_y = [d[0] for i, d in enumerate(x_vals) if y_vals[i] == -1]
train_indices = np.random.choice(len(x_vals), round(len(x_vals) * 0.8), replace=False)
test_indices = np.array(list(set(range(len(x_vals))) - set(train_indices)))
x_vals_train = x_vals[train_indices]
x_vals_test = x_vals[test_indices]
y_vals_train = y_vals[train_indices]
y_vals_test = y_vals[test_indices]

model = keras.Sequential([layers.Dense(1, input_shape=(2,), activation="linear")])

model.compile(
    optimizer=keras.optimizers.legacy.SGD(learning_rate=0.01),
    loss="hinge",
    metrics=["accuracy"],
)

history = model.fit(
    x_vals_train,
    y_vals_train,
    batch_size=100,
    epochs=500,
    validation_data=(x_vals_test, y_vals_test),
)

# Export model in TensorFlow's .pb format with 'serving_default' signature
model.save("exported_model")

# Convert the TensorFlow model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model("exported_model")
tflite_model = converter.convert()

## Convert model to msgpack using ZKML

In [ ]:
converter = Converter(model_path=TFLITE_PATH, expose_output=False, commit=False, scale_factor=1, k=9, num_cols=2, num_randoms=6, use_selectors=True)

model_packed, config_packed = converter.to_msgpack(
  start_layer=0,
  end_layer=10000,
)
if model_packed is None:
  raise Exception('Failed to convert model')
with open(OUT_MODEL_PATH, 'wb') as f:
  f.write(model_packed)
with open(OUT_CONFIG_PATH, 'wb') as f:
  f.write(config_packed)

## Create msgpack from input

In [ ]:
with open (OUT_INPUT_NPY_PATH, 'wb') as f:
    np.save(f, np.array(x_vals[0]).reshape(-1, 1))

!python {INPUT_CONVERTER_PATH} --model_config {OUT_MODEL_PATH} --inputs {OUT_INPUT_NPY_PATH} --output {OUT_INPUT_MSG_PATH}


## Generate the proof

In [ ]:
!../../src/zkml/target/release/time_circuit {OUT_MODEL_PATH} {OUT_INPUT_MSG_PATH} kzg
#!../../src/zkml/target/release/test_circuit {OUT_MODEL_PATH} {OUT_INPUT_MSG_PATH} kzg
